In [58]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [59]:
pip install "datasets<4.0.0"


Note: you may need to restart the kernel to use updated packages.


In [60]:
from datasets import load_dataset

ds = load_dataset("keremberke/chest-xray-classification", "full")

In [61]:
print(ds)

DatasetDict({
    train: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 4077
    })
    validation: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 1165
    })
    test: Dataset({
        features: ['image_file_path', 'image', 'labels'],
        num_rows: 582
    })
})


In [62]:
example = ds["train"][1]
example.keys()          # check fields, usually 'image', 'label'
example["image"]        # should be a PIL image
example["image_file_path"]        # 0 or 1 (NORMAL / PNEUMONIA)


'/storage/hf-datasets-cache/all/datasets/60340657865253-config-parquet-and-info-keremberke-chest-xray-cla-9d66ea8b/downloads/extracted/8202f7dd6f1edf5e674abe75990eb233fbbca4408e132a3acd5268bd99708e15/NORMAL/IM-0006-0001_jpeg.rf.75b1553d7514f898d58eccd723dcf0d3.jpg'

In [63]:
import pandas as pd

train_labels = [ex["labels"] for ex in ds["train"]]
pd.Series(train_labels).value_counts(normalize=True)


1    0.729213
0    0.270787
Name: proportion, dtype: float64

In [64]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision.models import resnet18, ResNet18_Weights

from datasets import load_dataset

import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import (
    confusion_matrix,
    classification_report,
    roc_auc_score,
    accuracy_score,
    precision_score,
    recall_score
)
from tqdm import tqdm


In [65]:
train_labels = [x["labels"] for x in ds["train"]]
val_labels   = [x["labels"] for x in ds["validation"]]
test_labels  = [x["labels"] for x in ds["test"]]

def count_labels(labels):
    unique, counts = np.unique(labels, return_counts=True)
    return dict(zip(unique, counts))

print("Train:", count_labels(train_labels))
print("Validation:", count_labels(val_labels))
print("Test:", count_labels(test_labels))


Train: {0: 1104, 1: 2973}
Validation: {0: 304, 1: 861}
Test: {0: 171, 1: 411}


In [66]:
IMG_SIZE = 224

train_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],   # ImageNet stats
        std=[0.229, 0.224, 0.225],
    ),
])

eval_transform = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Lambda(lambda x: x.repeat(3, 1, 1) if x.shape[0] == 1 else x),
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    ),
])


In [67]:
def make_transform_fn(tform):
    def transform(batch):
        # batch["image"] is a list of PIL images
        batch["pixel_values"] = [tform(img) for img in batch["image"]]
        return batch
    return transform

ds_train = ds["train"].with_transform(make_transform_fn(train_transform))
ds_val   = ds["validation"].with_transform(make_transform_fn(eval_transform))
ds_test  = ds["test"].with_transform(make_transform_fn(eval_transform))


In [68]:
def collate_fn(batch):
    # batch: list of dicts
    pixel_values = torch.stack([item["pixel_values"] for item in batch])
    labels = torch.tensor([item["labels"] for item in batch], dtype=torch.long)
    return pixel_values, labels

BATCH_SIZE = 32

train_loader = DataLoader(ds_train, batch_size=BATCH_SIZE, shuffle=True,  collate_fn=collate_fn)
val_loader   = DataLoader(ds_val,   batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)
test_loader  = DataLoader(ds_test,  batch_size=BATCH_SIZE, shuffle=False, collate_fn=collate_fn)


In [69]:
num_normal = train_labels.count(0)
num_pneu   = train_labels.count(1)
print("NORMAL:", num_normal, " PNEUMONIA:", num_pneu)

# inverse frequency weights
class_weights = torch.tensor([1/num_normal, 1/num_pneu], dtype=torch.float)
print("Class weights:", class_weights)


NORMAL: 1104  PNEUMONIA: 2973
Class weights: tensor([0.0009, 0.0003])


In [70]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

weights = ResNet18_Weights.IMAGENET1K_V1
model = resnet18(weights=weights)

num_ftrs = model.fc.in_features
model.fc = nn.Linear(num_ftrs, 2)   # 2 classes: NORMAL, PNEUMONIA

model = model.to(device)


Using device: cuda


In [71]:
class_weights = class_weights.to(device)
criterion = nn.CrossEntropyLoss(weight=class_weights)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-4)

scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optimizer,
    mode='min',
    factor=0.5,
    patience=2,
    verbose=True
)


/usr/local/lib/python3.11/dist-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


In [72]:
def train_model(model, train_loader, val_loader, criterion, optimizer, scheduler, device, epochs=10):
    model.to(device)

    for epoch in range(epochs):
        print(f"\nEpoch {epoch+1}/{epochs}")
        print("-" * 30)

        # ----- TRAIN -----
        model.train()
        train_loss = 0.0
        train_preds = []
        train_labels_all = []

        for x, y in tqdm(train_loader):
            x, y = x.to(device), y.to(device)

            optimizer.zero_grad()
            logits = model(x)
            loss = criterion(logits, y)
            loss.backward()
            optimizer.step()

            train_loss += loss.item()

            preds = torch.argmax(logits, dim=1)
            train_preds.extend(preds.cpu().numpy())
            train_labels_all.extend(y.cpu().numpy())

        train_loss /= len(train_loader)
        train_acc  = accuracy_score(train_labels_all, train_preds)
        train_prec = precision_score(train_labels_all, train_preds, zero_division=0)
        train_rec  = recall_score(train_labels_all, train_preds, zero_division=0)

        # ----- VALIDATION -----
        model.eval()
        val_loss = 0.0
        val_preds = []
        val_labels_all = []

        with torch.no_grad():
            for x, y in val_loader:
                x, y = x.to(device), y.to(device)
                logits = model(x)
                loss = criterion(logits, y)

                val_loss += loss.item()

                preds = torch.argmax(logits, dim=1)
                val_preds.extend(preds.cpu().numpy())
                val_labels_all.extend(y.cpu().numpy())

        val_loss /= len(val_loader)
        val_acc  = accuracy_score(val_labels_all, val_preds)
        val_prec = precision_score(val_labels_all, val_preds, zero_division=0)
        val_rec  = recall_score(val_labels_all, val_preds, zero_division=0)

        scheduler.step(val_loss)

        print(f"Train Loss: {train_loss:.4f}")
        print(f"Train Acc:  {train_acc:.4f} | Prec: {train_prec:.4f} | Rec: {train_rec:.4f}")
        print(f"Val   Loss: {val_loss:.4f}")
        print(f"Val   Acc:  {val_acc:.4f} | Prec: {val_prec:.4f} | Rec: {val_rec:.4f}")

    print("\nTraining complete.")
    return model


In [73]:
EPOCHS = 10   # increase to 15–20 if using GPU
model = train_model(
    model,
    train_loader,
    val_loader,
    criterion,
    optimizer,
    scheduler,
    device,
    epochs=EPOCHS
)



Epoch 1/10
------------------------------


100%|██████████| 128/128 [00:34<00:00,  3.70it/s]


Train Loss: 0.1852
Train Acc:  0.9299 | Prec: 0.9722 | Rec: 0.9304
Val   Loss: 0.0884
Val   Acc:  0.9682 | Prec: 0.9905 | Rec: 0.9663

Epoch 2/10
------------------------------


100%|██████████| 128/128 [00:34<00:00,  3.72it/s]


Train Loss: 0.1045
Train Acc:  0.9627 | Prec: 0.9822 | Rec: 0.9664
Val   Loss: 0.1274
Val   Acc:  0.9562 | Prec: 0.9939 | Rec: 0.9466

Epoch 3/10
------------------------------


...

Train Loss: 0.0419
Train Acc:  0.9833 | Prec: 0.9929 | Rec: 0.9842
Val   Loss: 0.0871
Val   Acc:  0.9700 | Prec: 0.9847 | Rec: 0.9744

Epoch 9/10
------------------------------


100%|██████████| 128/128 [00:34<00:00,  3.71it/s]


Train Loss: 0.0505
Train Acc:  0.9774 | Prec: 0.9908 | Rec: 0.9781
Val   Loss: 0.0852
Val   Acc:  0.9691 | Prec: 0.9893 | Rec: 0.9686

Epoch 10/10
------------------------------


100%|██████████| 128/128 [00:34<00:00,  3.72it/s]


Train Loss: 0.0284
Train Acc:  0.9924 | Prec: 0.9970 | Rec: 0.9926
Val   Loss: 0.0653
Val   Acc:  0.9768 | Prec: 0.9871 | Rec: 0.9814

Training complete.


In [74]:
model.eval()
all_preds = []
all_labels = []
all_probs = []

with torch.no_grad():
    for x, y in test_loader:
        x, y = x.to(device), y.to(device)
        logits = model(x)
        probs = torch.softmax(logits, dim=1)[:, 1]  # P(PNEUMONIA)
        preds = torch.argmax(logits, dim=1)

        all_labels.extend(y.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())
        all_probs.extend(probs.cpu().numpy())

cm = confusion_matrix(all_labels, all_preds)
print("Confusion Matrix:\n", cm)
print("\nClassification Report:\n",
      classification_report(all_labels, all_preds, target_names=["NORMAL", "PNEUMONIA"]))
print("ROC-AUC:", roc_auc_score(all_labels, all_probs))


Confusion Matrix:
 [[163   8]
 [  7 404]]

Classification Report:
               precision    recall  f1-score   support

      NORMAL       0.96      0.95      0.96       171
   PNEUMONIA       0.98      0.98      0.98       411

    accuracy                           0.97       582
   macro avg       0.97      0.97      0.97       582
weighted avg       0.97      0.97      0.97       582

ROC-AUC: 0.9959163927661814


In [75]:
from PIL import Image

In [82]:
from PIL import Image
import numpy as np
import torch

# Class index → label
idx_to_label = {0: "NORMAL", 1: "PNEUMONIA"}

def predict_image(image_path):
    """
    Predict class + probabilities for a single chest X-ray image.
    """
    # 1. Load image (X-rays are grayscale)
    img = Image.open(image_path).convert("L")

    # 2. Apply SAME eval_transform as for test_loader
    x = eval_transform(img)          # [3, H, W] after your Lambda
    x = x.unsqueeze(0).to(device)    # [1, 3, H, W]

    # 3. Forward pass
    model.eval()
    with torch.no_grad():
        logits = model(x)
        probs = torch.softmax(logits, dim=1).cpu().numpy()[0]

    # 4. Get predicted class + prob
    pred_idx = int(np.argmax(probs))
    pred_label = idx_to_label[pred_idx]
    pred_prob = float(probs[pred_idx])

    print(f"Image: {image_path}")
    print(f"Prediction: {pred_label} (class {pred_idx})")
    print(f"Confidence: {pred_prob:.4f}")
    print(f"Class probabilities → NORMAL={probs[0]:.4f}, PNEUMONIA={probs[1]:.4f}")

    return pred_label, pred_prob, probs


In [83]:
predict_image('/kaggle/input/chest-xray/image.jpg')

Image: /kaggle/input/chest-xray/image.jpg
Prediction: PNEUMONIA (class 1)
Confidence: 1.0000
Class probabilities → NORMAL=0.0000, PNEUMONIA=1.0000


('PNEUMONIA',
 0.9999997615814209,
 array([2.2406127e-07, 9.9999976e-01], dtype=float32))